In [9]:
#importing libraries
import time
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from selenium import webdriver
from selenium.webdriver.common.by import By
import networkx as nx

In [32]:
driver = webdriver.Chrome()


driver.get("https://www.jpost.com/breaking-news") 

time.sleep(5)
# Find all links

links = driver.find_elements(By.CSS_SELECTOR, 'a')

url_list = []
for link in links:
    url_list.append(link.get_attribute('href'))
    
G = nx.Graph()

for url in url_list:
    driver.get(url)
    article_text = driver.find_element(By.CSS_SELECTOR, 'a').text
    
    source_links = driver.find_elements(By.CSS_SELECTOR, 'a.source')
    for source in source_links:
        source_url = source.get_attribute('href')
        G.add_edge(url, source_url)

nx.draw(G)
print(links)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a"}
  (Session info: chrome=119.0.6045.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D41582B2+55298]
	(No symbol) [0x00007FF6D40C5E02]
	(No symbol) [0x00007FF6D3F805AB]
	(No symbol) [0x00007FF6D3FC175C]
	(No symbol) [0x00007FF6D3FC18DC]
	(No symbol) [0x00007FF6D3FFCBC7]
	(No symbol) [0x00007FF6D3FE20EF]
	(No symbol) [0x00007FF6D3FFAAA4]
	(No symbol) [0x00007FF6D3FE1E83]
	(No symbol) [0x00007FF6D3FB670A]
	(No symbol) [0x00007FF6D3FB7964]
	GetHandleVerifier [0x00007FF6D44D0AAB+3694587]
	GetHandleVerifier [0x00007FF6D452728E+4048862]
	GetHandleVerifier [0x00007FF6D451F173+4015811]
	GetHandleVerifier [0x00007FF6D41F47D6+695590]
	(No symbol) [0x00007FF6D40D0CE8]
	(No symbol) [0x00007FF6D40CCF34]
	(No symbol) [0x00007FF6D40CD062]
	(No s